# SQL 的五十道練習：初學者友善的資料庫入門

> 函數

In [1]:
%LOAD sqlite3 db=../databases/imdb.db timeout=2 shared_cache=true

In [2]:
ATTACH "../databases/covid19.db" AS covid19;

## 複習一下

在第三章「從資料表選擇」我們將 SQL 敘述歸納為以下幾個部分的組成：

- 保留字：具有特定功能的指令。
- 符號：具有特定功能的符號。
- 常數：由使用者給予的資料。
- 函數：具有特定邏輯的輸入與輸出對應。

其中在「函數」這個部分，我們可以使用函數來對不同資料類別應用，包含整數、浮點數、文字與空值，藉此讓輸入經由特定邏輯處理後對應為輸出，函數不僅能應用於常數之上，亦能應用在資料表的欄。

## 函數

Function，中文翻譯為函數或者函式，在資料分析和程式語言中都扮演舉足輕重的角色！函數是預先被定義好的運算處理邏輯，透過它的作用，能夠將「輸入」對應為「輸出」，進而完成計算數值、操作文字以及資料類別相關等任務。函數可以粗略分為兩大類：

1. 通用函數（Universal functions）。
2. 聚合函數（Aggregate functions）。

其中通用函數又能細分為四種：

1. 資料類別相關。
2. 計算數值。
3. 操作文字。
4. 操作日期時間。

理解函數的運作首先我們要認識函數是怎麼組成的，函數由「函數的名稱」、「輸入」、「參數」、「運算處理邏輯」以及「輸出」所組成，以日常生活中去手搖飲料店買珍珠奶茶來比喻會更容易理解：

- 函數的名稱：去手搖飲料店買珍珠奶茶。
- 輸入（Input）：珍珠奶茶的價錢。
- 參數（Parameter）：甜度以及冰度。
- 運算處理邏輯：從訂單成立、製作珍珠奶茶、封口最後是成品。
- 輸出（Output）：依據價錢、甜度與冰度所製作的珍珠奶茶。

![](../images/rosalind-chang-P_wPicZYoPI-unsplash.jpg)

來源：[Photo by Rosalind Chang on Unsplash](https://unsplash.com/photos/P_wPicZYoPI)

使用函數的語法是：

```sql
SELECT FUNCTION(input, parameter) AS alias
```

以一個常用的數值運算函數 `ROUND()` 為例說明，`ROUND()` 函數能夠對輸入的浮點數四捨五入至指定小數點位數：

In [3]:
SELECT 2.718 AS e,
       ROUND(2.718) AS round_e_0,    -- round to 0 digit
       ROUND(2.718, 1) AS round_e_1, -- round to 1 digit
       ROUND(2.718, 2) AS round_e_2; -- round to 2 digits

e,round_e_0,round_e_1,round_e_2
2.718,3.0,2.7,2.72


再以另一個常用的文字操作函數 `SUBSTR()` 為例說明，`SUBSTR()` 函數能夠對輸入的文字從指定起點擷取指定長度的字串：

In [4]:
SELECT 'Tony Stark' AS ironman,
       SUBSTR('Tony Stark', 1, 4) AS first_name, -- sub-string from the 1st character for length of 4
       SUBSTR('Tony Stark', 6, 5) AS last_name;  -- sub-string from the 6th character for length of 5

ironman,first_name,last_name
Tony Stark,Tony,Stark


函數中的輸入與參數有時不一定需要指定，有些函數的設計不需要輸入，例如 `DATE()` 函數能夠回傳電腦當下的日期；有些函數的設計參數具有預設（Default），如果沒有指定就採用預設，例如 `ROUND()` 函數如果沒有指定四捨五入至幾位小數，則會採用 `0` 為預設，意即四捨五入到整數位數、小數位數 `0`。

```sql
SELECT ROUND(2.718) AS round_e_0,
       DATE() AS todays_date;
```

使用函數還有一點值得注意的概念：複合函數（Composite functions），意即在函數中包括函數、先後使用多個函數，先使用的函數輸出將會成為後使用的函數輸入。舉例來說，`SUBSTR()` 函數的輸出為 `'Bos'`，成為 `UPPER()` 函數的輸入，最後的輸出為 `'BOS'`。

In [5]:
SELECT 'Boston' AS city,
       UPPER(SUBSTR('Boston', 1, 3)) AS composite_function;

city,composite_function
Boston,BOS


## 通用函數與聚合函數

前一個小節我們提到函數可以粗略分為兩大類：通用函數（Universal functions）與聚合函數（Aggregate functions），之所以區分為兩類，是因為函數的作用方向不同，跟資料表是由列（水平方向）與欄（垂直方向）所組成的二維表格概念契合，通用函數作用在「水平方向」、聚合函數作用在「垂直方向」。具體來說，通用函數的特徵是每列觀測值對應一個輸出結果，效果類似「衍生計算欄位」，差別在於一個是以函數輸出衍生計算欄位，一個則是以運算符生成衍生計算欄位，例如常用的數值運算函數 `ROUND()` 就是一個通用函數。

In [6]:
SELECT rating,
       ROUND(rating) AS round_rating
  FROM movies
 LIMIT 5;

rating,round_rating
9.3,9.0
9.2,9.0
9,9.0
9,9.0
9,9.0


常用的文字操作函數 `SUBSTR()` 同樣也是一個通用函數。

In [7]:
SELECT title,
       SUBSTR(title, 1, 3) AS substr_title
  FROM movies
 LIMIT 4;

title,substr_title
The Shawshank Redemption,The
The Godfather,The
The Dark Knight,The
The Godfather Part II,The


聚合函數的特徵是一欄變數、`m` 列觀測值對應一個輸出結果，例如常用於摘要數值的函數 `AVG()` 就是一個通用函數，`AVG()` 函數能夠對輸入的數值欄位取其平均值。

In [8]:
SELECT AVG(rating) AS avg_rating
  FROM movies;

avg_rating
8.30719999999998


聚合函數的特徵是一欄變數、`m` 列觀測值對應一個輸出結果，例如常用於摘要數值的函數 `AVG()` 就是一個通用函數，`AVG()` 函數能夠對輸入的數值欄位取其平均值。

## 通用函數

每列觀測值對應一個輸出結果的通用函數又可以細分為四種類型：

1. 資料類別相關。
2. 計算數值。
3. 操作文字。
4. 操作日期時間。

我們從 SQLite 的官方文件 <https://www.sqlite.org/lang_corefunc.html> 挑出部分函數跟讀者簡介，不會一一示範，讀者只要理解函數的使用語法：

```sql
SELECT FUNCTION(input, parameter) AS alias
```

以及本章少數的函數使用範例，應該能夠舉一反三。

### 通用函數：資料類別相關 `TYPEOF()`

使用 `TYPEOF()` 函數顯示常數或者資料表欄位的類別。

In [9]:
SELECT TYPEOF(release_year) AS typeof_release_year,
       TYPEOF(rating) AS typeof_rating,
       TYPEOF(title) AS typeof_title
  FROM movies
 LIMIT 1;

typeof_release_year,typeof_rating,typeof_title
integer,real,text


### 通用函數：資料類別相關 `IFNULL()`

使用 `IFNULL()` 函數回傳輸入常數或者資料表欄位中的第一個非空值資料，若皆為空值則回傳空值。

In [10]:
SELECT IFNULL(NULL, NULL) AS null_value,
       IFNULL(NULL, 'Null replaced by text') AS text_value;

null_value,text_value
NULL,Null replaced by text


舉例來說，在 `covid19` 資料庫的 `lookup_table` 資料表中 `Province_State` 與 `Admin2` 欄都有空值的存在，如果對這兩欄分別使用 `IFNULL()` 函數，可以將空值取代為指定文字。

In [11]:
SELECT Province_State,
       IFNULL(Province_State, 'No province data') AS province_or_text_value,
       Admin2,
       IFNULL(Admin2, 'No county data') AS county_or_text_value
  FROM lookup_table
 LIMIT 5;

Province_State,province_or_text_value,Admin2,county_or_text_value
NULL,No province data,NULL,No county data
NULL,No province data,NULL,No county data
NULL,No province data,NULL,No county data
NULL,No province data,NULL,No county data
American Samoa,American Samoa,NULL,No county data


### 通用函數：資料類別相關 `COALESCE()`

使用 `COALESCE()` 函數回傳輸入常數或者資料表欄位中的第一個非空值資料，若皆為空值則回傳空值，與 `IFNULL()` 不同的地方在於可以接受兩個以上的輸入。

In [12]:
SELECT COALESCE(NULL, NULL) AS null_value,
       COALESCE(NULL, 'Null replaced by text') AS text_value,
       COALESCE(NULL, NULL, 'Null replaced by text') AS text_value;

null_value,text_value,text_value
NULL,Null replaced by text,Null replaced by text


舉例來說，在 `covid19` 資料庫的 `lookup_table` 資料表中 `Province_State` 與 `Admin2` 欄都有空值的存在，如果對這兩欄使用 `COALESCE()` 函數，可以在僅有 `Admin2` 為空值的列數回傳 `Province_State` 值；在 `Province_State` 與 `Admin2` 皆為空值的列數回傳指定文字。

In [13]:
SELECT Province_State,
       Admin2,
       COALESCE(Province_State, Admin2) AS province_or_null,
       COALESCE(Province_State, Admin2, 'No province or county data') AS province_or_text_value
  FROM lookup_table
 LIMIT 5;

Province_State,Admin2,province_or_null,province_or_text_value
NULL,NULL,NULL,No province or county data
NULL,NULL,NULL,No province or county data
NULL,NULL,NULL,No province or county data
NULL,NULL,NULL,No province or county data
American Samoa,NULL,American Samoa,American Samoa


### 通用函數：計算數值 `ROUND()`

`ROUND()` 函數能夠對輸入的浮點數四捨五入至指定小數點位數。

In [14]:
SELECT runtime,
       ROUND(runtime/60.0) AS hours_0,    -- round to 0 digit
       ROUND(runtime/60.0, 1) AS hours_1, -- round to 1 digit
       ROUND(runtime/60.0, 2) AS hours_2  -- round to 2 digits
  FROM movies
 LIMIT 5;

runtime,hours_0,hours_1,hours_2
142,2.0,2.4,2.37
175,3.0,2.9,2.92
152,3.0,2.5,2.53
202,3.0,3.4,3.37
96,2.0,1.6,1.6


### 通用函數：操作文字 `LENGTH()`

`LENGTH()` 函數可以計算輸入文字資料類別中有幾個字元，包含空格以及標點符號。

In [15]:
SELECT title,
       LENGTH(title) AS length_title
  FROM movies
 LIMIT 5;

title,length_title
The Shawshank Redemption,24
The Godfather,13
The Dark Knight,15
The Godfather Part II,21
12 Angry Men,12


### 通用函數：操作文字 `SUBSTR()`

`SUBSTR()` 函數可利用位置與長度將輸入文字的指定段落擷取出來。

In [16]:
SELECT title,
       SUBSTR(title, 1, 3) AS substr_title
  FROM movies
 LIMIT 4;

title,substr_title
The Shawshank Redemption,The
The Godfather,The
The Dark Knight,The
The Godfather Part II,The


###  通用函數：操作文字 `LOWER()` 與 `UPPER()`

`LOWER()` 與 `UPPER()` 函數可以調整英文的大小寫。

In [17]:
SELECT title,
       LOWER(title) AS lower_title,
       UPPER(title) AS upper_title
  FROM movies
 LIMIT 5;

title,lower_title,upper_title
The Shawshank Redemption,the shawshank redemption,THE SHAWSHANK REDEMPTION
The Godfather,the godfather,THE GODFATHER
The Dark Knight,the dark knight,THE DARK KNIGHT
The Godfather Part II,the godfather part ii,THE GODFATHER PART II
12 Angry Men,12 angry men,12 ANGRY MEN


### 通用函數：操作日期時間

`DATE('now', 'localtime')`、`TIME('now', 'localtime')` 與 `DATETIME('now', 'localtime')` 函數可以顯示電腦時區現在的日期、時間與日期時間，並且以 ISO-8601 標準格式 `YYYY-MM-DD HH:MM:SS` 呈現

```sql
SELECT DATE('now', 'localtime') AS date_now,
       TIME('now', 'localtime') AS time_now,
       DATETIME('now', 'localtime') AS datetime_now;
```

我們也可以輸入 ISO-8601 標準格式作為日期、時間或日期時間。

In [18]:
SELECT DATE('2022-12-31') AS date_20221231,
       TIME('23:59:59') AS time_235959,
       DATETIME('2022-12-31 23:59:59') AS datetime_20221231235959;

date_20221231,time_235959,datetime_20221231235959
2022-12-31,23:59:59,2022-12-31 23:59:59


### 通用函數：操作日期時間 `STRFTIME()`

`STRFTIME()` 函數可以調整日期、時間或日期時間的顯示格式，常用的日期、時間格式參數有：

- `%d`：二位數的日（01-31）
- `%j`：一年中的第幾天（001-366）
- `%m`：二位數的月（01-12）
- `%w`：一星期中的第幾天（0-6）
- `%W`：一年中的第幾週（00-53）
- `%Y`：四位數的年（0000-9999）
- `%H`：兩位數的小時（00-24）
- `%M`：兩位數的分鐘（00-59）
- `%S`：兩位數的秒（00-59）

In [19]:
SELECT '2022-12-31 23:59:59' AS datetime_20221231235959,
       STRFTIME('%d', '2022-12-31 23:59:59') AS day_part,
       STRFTIME('%j', '2022-12-31 23:59:59') AS day_of_year,
       STRFTIME('%m', '2022-12-31 23:59:59') AS month_part,
       STRFTIME('%w', '2022-12-31 23:59:59') AS weekday,
       STRFTIME('%W', '2022-12-31 23:59:59') AS nth_week,
       STRFTIME('%Y', '2022-12-31 23:59:59') AS year_part,
       STRFTIME('%H:%M:%S', '2022-12-31 23:59:59') AS time_part;

datetime_20221231235959,day_part,day_of_year,month_part,weekday,nth_week,year_part,time_part
2022-12-31 23:59:59,31,365,12,6,52,2022,23:59:59


## 聚合函數

一欄變數、`m` 列觀測值對應一個輸出結果的常用聚合函數有：

- `AVG()`：計算欄位的平均數。
- `COUNT(*)`：計算資料表的列數。
- `COUNT(column)`：計算欄位的「非空值」列數。
- `MIN()`：計算欄位的最小值。
- `MAX()`：計算欄位的最大值。
- `SUM()`：計算欄位的加總。

聚合函數的名稱和其中所含的運算處理邏輯是相當直觀的，例如 `AVG()` 作用即為 average、`MIN()` 作用即為 minimum、`MAX()` 作用即為 maximum。

In [20]:
SELECT AVG(rating) AS avg_rating,
       MIN(rating) AS min_rating,
       MAX(rating) AS max_rating
  FROM movies;

avg_rating,min_rating,max_rating
8.30719999999998,8.0,9.3


使用聚合函數 `COUNT()` 時候要注意兩種不同的用法，其一是輸入 `*` 藉以獲得資料表列數；另一則是輸入欄位的名稱獲得該欄位「非空值」列數，這兩種不同用法在沒有空值 `NULL` 的欄位上輸出是相同的，但是對於有空值 `NULL` 的欄位，就能夠發現差異之處。舉例來說，在 `covid19` 資料庫的 `lookup_table` 資料表中 `Province_State` 與 `Admin2` 欄都有空值的存在，如果對這兩欄使用 `COUNT()` 函數會得到與 `COUNT(*)` 不同的輸出結果；反之 `lookup_table` 資料表中 `Combined_Key` 欄沒有空值的存在，對它使用 `COUNT()` 函數會得到與 `COUNT(*)` 相同的輸出結果。

In [21]:
SELECT COUNT(*) AS nrows_table,
       COUNT(Province_State) AS non_null_province,
       COUNT(Admin2) AS non_null_county
  FROM lookup_table;

nrows_table,non_null_province,non_null_county
4317,4118,3344


In [22]:
SELECT COUNT(*) AS nrows_table,
       COUNT(Combined_Key) AS non_null_combined_key
  FROM lookup_table;

nrows_table,non_null_combined_key
4317,4317


## 重點統整

- 需要特定函數處理資料類別相關、數值計算、文字操作或日期時間操作時，可以查閱 SQLite 函數官方文件：
    - 通用函數：<https://www.sqlite.org/lang_corefunc.html>
    - 日期時間操作函數：<https://www.sqlite.org/lang_datefunc.html>
    - 聚合函數：<https://www.sqlite.org/lang_aggfunc.html>
- 將截至目前所學的 SQL 保留字集中在一個敘述中，寫作順序必須遵從標準 SQL 的規定。

```sql
SELECT DISTINCT columns AS alias
  FROM table
 LIMIT m;
```

## 練習題 09-13

練習題會涵蓋四個學習資料庫，記得要依據題目的需求，調整編輯器選單的學習資料庫，在自己電腦的 SQLiteStudio 寫出跟預期輸出相同的 SQL 敘述，寫作過程如果卡關了，可以參考附錄二「練習題參考解答」。

### 09. 從 `nba` 資料庫的 `players` 資料表依據 `heightMeters`、`weightKilograms` 以及下列公式衍生計算欄位 `bmi`，並使用 `ROUND` 函數將 `bmi` 的小數點位數調整為 2 位，參考下列的預期查詢結果。

\begin{equation}
BMI = \frac{weight_{kg}}{height_{m}^2}
\end{equation}

預期輸出：(506, 3) 的查詢結果。

```
heightMeters	weightKilograms	bmi
2.06	113.4	26.72
2.01	108	26.73
2.03	106.6	25.87
2.08	120.2	27.78
1.98	97.5	24.87
...
```

### 10. 從 `nba` 資料庫的 `career_summaries` 資料表中依據 `assists`、`turnovers` 欄位以及下列公式衍生計算助攻失誤比，讓衍生計算欄位的資料類型為浮點數 `REAL`，參考下列的預期查詢結果。

\begin{equation}
\text{Assists Turnover Ratio} = \frac{\text{Assists}}{\text{Turnovers}}
\end{equation}

預期輸出：(506, 3) 的查詢結果。

```
assists	turnovers	assist_turnover_ratio
10045	4788	2.09795321637427
3422	3052	1.12123197903014
733	809	0.90605686032138
1676	3302	0.50757116898849
5128	2231	2.29852084267145
...
```

### 11. 從 `covid19` 資料庫的 `time_series` 資料表依據 `Date` 變數，使用 `STRFTIME` 函數查詢時間序列資料有哪些不重複的月份，參考下列的預期查詢結果。

預期輸出：(29, 1) 的查詢結果。

```
distinct_year_month
2020-01
2020-02
2020-03
2020-04
2020-05
2020-06
2020-07
2020-08
2020-09
2020-10
2020-11
2020-12
2021-01
2021-02
2021-03
2021-04
2021-05
2021-06
2021-07
2021-08
2021-09
2021-10
2021-11
2021-12
2022-01
2022-02
2022-03
2022-04
2022-05
```

### 12. 從 `twElection2020` 資料庫的 `presidential` 資料表利用聚合函數彙總有多少人參與了總統副總統的投票選舉，參考下列的預期查詢結果。

預期輸出：(1, 1) 的查詢結果。

```
total_presidential_votes
14300940
```

### 13. 從 `covid19` 資料庫的 `daily_report` 資料表利用聚合函數彙總截至 2022-05-31 全世界總確診數以及總死亡數，參考下列的預期查詢結果。

註：本題不需考慮 `daily_report` 內的 `Last_Update` 時間戳記，`daily_report` 的數據有效期間就是 2022-05-31。

預期輸出：(1, 2) 的查詢結果。

```
total_confirmed	total_deaths
529625234	6292512
```